Derive spec2vec embeddings of MS/MS spectra

In [17]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    import os
import sys
import gensim
import numpy as np

ROOT = os.path.dirname(os.getcwd())
#path_data = os.path.join(ROOT, 'data')
path_data = 'C:\\Users\\Gosia\\Desktop\\'
sys.path.insert(0, ROOT)

In [18]:
from matchms.importing import load_from_json
spectrums_lib = []
path_lcms = 'C:\\Users\\Gosia\\Desktop\\gnps_from_simon'
counter = 0
for s in os.listdir(path_lcms):
    if counter <= 5: 
        spectrums_lib += load_from_json(os.path.join(path_lcms,s))
        counter += 1

In [19]:
from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.filtering import select_by_relative_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import add_losses
def post_process_s2v(s):
    
    s = normalize_intensities(s)
    s = select_by_mz(s, mz_from=0, mz_to=1000)
    s = require_minimum_number_of_peaks(s, n_required=10)
    s = reduce_to_number_of_peaks(s, n_required=10, ratio_desired=0.5)
    if s is None:
        return None
    s_remove_low_peaks = select_by_relative_intensity(s, intensity_from=0.001)
    if len(s_remove_low_peaks.peaks) >= 10:
        s = s_remove_low_peaks
        
    s = add_losses(s, loss_mz_from=5.0, loss_mz_to=200.0)
    return s

In [20]:
# apply post processing steps to the data

spectrums_lib = [post_process_s2v(s) for s in spectrums_lib]

# omit spectrums that didn't qualify for analysis

spectrums_lib = [s for s in spectrums_lib if s is not None]


Create spectrum "documents"

In [32]:
from spec2vec import Spec2Vec
from spec2vec import SpectrumDocument

documents_lib = [SpectrumDocument(s, n_decimals=2) for s in spectrums_lib]


In [64]:
from spec2vec.model_building import train_new_word2vec_model
path_models = os.path.join(path_data, "trained_models")

model_file = os.path.join(path_models, "spec2vec_librarymatching_size_3.model")

iterations = [1, 3, 5, 10]

#Train model with size 10 and default parameters

model = train_new_word2vec_model(documents_lib, iterations, model_file, size = 3)

The value of size is set from 300 (default) to 3
  Epoch 1 of 10.Change in loss after epoch 1: 125458.3046875
Saving model with name: C:\Users\Gosia\Desktop\trained_models\spec2vec_librarymatching_size_3_iter_1.model
  Epoch 2 of 10.Change in loss after epoch 2: 125569.4609375
  Epoch 3 of 10.Change in loss after epoch 3: 122548.109375
Saving model with name: C:\Users\Gosia\Desktop\trained_models\spec2vec_librarymatching_size_3_iter_3.model
  Epoch 4 of 10.Change in loss after epoch 4: 99907.4375
  Epoch 5 of 10.Change in loss after epoch 5: 87899.1875
Saving model with name: C:\Users\Gosia\Desktop\trained_models\spec2vec_librarymatching_size_3_iter_5.model
  Epoch 6 of 10.Change in loss after epoch 6: 86848.1875
  Epoch 7 of 10.Change in loss after epoch 7: 83071.8125
  Epoch 8 of 10.Change in loss after epoch 8: 82865.375
  Epoch 9 of 10.Change in loss after epoch 9: 78945.1875
  Epoch 10 of 10.Change in loss after epoch 10: 79475.1875
Saving model with name: C:\Users\Gosia\Desktop\t


Derive embeddings

In [65]:
from tqdm.notebook import tqdm  # optional, just to get a progress bar
from spec2vec.vector_operations import calc_vector


intensity_weighting_power = 0.5
allowed_missing_percentage = 15 # specify the maximum (weighted) fraction of the spectrum that is allowed to be missing

vector_size = model.vector_size
print(f"Embedding vector size: {vector_size}")

embeddings_spec2vec_lib = np.zeros((len(documents_lib), vector_size), dtype="float")
for i, doc in enumerate(tqdm(documents_lib)):
    embeddings_spec2vec_lib[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power,
                                                        allowed_missing_percentage)

Embedding vector size: 3


In [66]:
print([np.round(x, 4) for x in embeddings_spec2vec_lib])

[array([ 218.7807,  -34.567 , -116.4491]), array([112.5963, -18.298 , -54.8105]), array([12.7538, -1.6625, -4.8881]), array([24.464 , -2.9445, -9.8389]), array([247.546 ,  75.0171, 158.6741]), array([ 49.8803,  -6.0715, -20.5648]), array([18.4992, -2.3365, -7.7398]), array([231.8018,  11.6846,  18.154 ]), array([20.5491, -2.0005, -6.0997]), array([ 6.8564, -1.0612, -3.0418]), array([232.2264,  84.0546,  99.9234]), array([ 7.0762, -0.5214, -2.4867]), array([19.7994, -1.2698, -4.4047]), array([45.2973,  0.4848,  9.1424]), array([107.5481, -15.278 , -42.825 ]), array([124.9474,  38.3459,  50.7101]), array([16.1681, -1.9753, -4.3677]), array([183.483 , -32.1062, -96.4969]), array([ 87.1373,   7.4409, -24.288 ]), array([35.37  , -3.6038, -5.0023]), array([ 79.0504,  -9.5536, -29.7316]), array([12.4807, -1.8627, -2.2861]), array([99.1034, 15.2044, 20.9403]), array([ 36.034 ,  -6.8517, -16.2751]), array([ 66.5917, -10.3628, -34.3864]), array([ 8.3042, -0.709 , -1.5659]), array([154.1077,  51.

In [96]:
embeddings_spec2vec_lib[:2]

array([[ 218.78073387,  -34.56697576, -116.44910337],
       [ 112.59629613,  -18.29796321,  -54.81049304]])

In [126]:
import numpy as np
from sklearn.mixture import GaussianMixture

gm = GaussianMixture().fit(embeddings_spec2vec_lib)
gm.means_
gm.covariances_[0]
np.dot(np.dot(D,np.linalg.inv(Sigma)),mu.T)
np.dot(A, point.T)
point,mu

(array([112.37254659,  -0.73752769,  -6.6526448 ]),
 array([123.22816548,   3.50904405,  -5.46250201]))

# Creating knockoffs


In [129]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import numpy.ma as ma
from numpy.random import uniform, seed
from matplotlib import cm
from scipy.stats import multivariate_normal

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

# seed(1234)
nDim = len(embeddings_spec2vec_lib[0])
# define the mean and covariance 
mu = gm.means_[0]
Sigma = gm.covariances_[0]
D = np.eye(nDim)*0.13

joint_cov = np.hstack((Sigma, Sigma-D))
joint_cov = np.vstack((joint_cov, np.hstack((Sigma-D,Sigma))))

print(is_pos_def(joint_cov))

A = np.eye(nDim) - np.dot(D,np.linalg.inv(Sigma))

all_knockoffs = []
# generate a sample
for point in embeddings_spec2vec_lib:
    # generate N knock-offs
    kmu = np.dot(np.dot(D,np.linalg.inv(Sigma)),mu)
    B = np.dot(A, point.T)
    kmu += B
    kSigma = 2*D - np.dot(np.dot(D,np.linalg.inv(Sigma)), D)
    ko = np.random.multivariate_normal(kmu.flatten(), kSigma, 1)
    all_knockoffs.append(ko)


for ko,v in zip(all_knockoffs,embeddings_spec2vec_lib):
    print("knockoff:",ko,"vector",v)
    print("\n")
    
print(all_knockoffs)

True
knockoff: [[ 219.47205991  -35.18952241 -117.04375941]] vector [ 218.78073387  -34.56697576 -116.44910337]


knockoff: [[112.80809725 -18.34455062 -54.94831533]] vector [112.59629613 -18.29796321 -54.81049304]


knockoff: [[12.21989165 -1.27862882 -4.35599765]] vector [12.75382666 -1.66247233 -4.88809043]


knockoff: [[25.58564814 -3.11077757 -9.1992328 ]] vector [24.46395276 -2.94450974 -9.83890058]


knockoff: [[247.07281057  74.23165735 159.40965625]] vector [247.54601796  75.01714128 158.67407735]


knockoff: [[ 51.08368006  -5.90128095 -20.07682393]] vector [ 49.88025681  -6.07150762 -20.56478523]


knockoff: [[18.19721824 -2.63663669 -8.12654407]] vector [18.49915793 -2.33649847 -7.73977271]


knockoff: [[232.59205736  11.80496693  17.76188628]] vector [231.80182604  11.68462686  18.15400574]


knockoff: [[21.61474151 -2.10936474 -6.12626773]] vector [20.54910556 -2.00049827 -6.09966942]


knockoff: [[ 7.51653477 -1.51648886 -2.48089026]] vector [ 6.85640981 -1.06119456 -3.0

knockoff: [[22.17450878 -2.09934951 -9.13260569]] vector [22.0428393  -1.54356478 -8.99139765]


knockoff: [[83.41860377  8.7786063  29.07583052]] vector [83.66670425  9.05743244 30.52816521]


knockoff: [[86.05364545 -1.98692622  8.20013604]] vector [85.9484396  -1.85679866  9.28732797]


knockoff: [[178.8507792   -3.13427682   8.67850546]] vector [178.39315918  -3.6243749    8.64574629]


knockoff: [[56.7727255   3.53884236 13.69730401]] vector [55.74607477  4.24016376 13.0851866 ]


knockoff: [[121.08367877   6.9711384   14.218042  ]] vector [120.97047051   5.90733013  13.53919637]


knockoff: [[49.07707798 -0.38697631  3.20874034]] vector [49.44909183 -0.23800659  3.10114364]


knockoff: [[18.00400504  7.98512012  2.04436668]] vector [17.42226535  7.78329724  2.15214601]


knockoff: [[50.48006587  4.71133614 19.81114045]] vector [50.20485418  5.64270233 19.96767196]


knockoff: [[42.28778779  4.86004594 11.63279453]] vector [43.2472244   3.75801096 12.3715945 ]


knockoff: [[12.760

knockoff: [[87.75372624 13.45434403 35.42656055]] vector [87.13678476 13.75112048 35.93729284]


knockoff: [[39.62252173  0.69748252 -2.86038053]] vector [39.35657016 -0.19932845 -3.02649395]


knockoff: [[184.43846307  28.42535901  16.99843325]] vector [184.20641046  28.73766994  16.84793828]


knockoff: [[148.95881583  24.29332275  20.72115464]] vector [149.1605654   24.37328572  20.65901273]


knockoff: [[103.84434146  -1.49994309 -14.60665304]] vector [103.62828571  -0.2879161  -14.27912103]


knockoff: [[27.68311907  7.2652799  12.47085436]] vector [27.83866504  6.87581696 13.01706623]


knockoff: [[78.71826733 16.63138982 28.02595854]] vector [78.54725491 17.02951565 27.20491106]


knockoff: [[97.26992013 18.37615722 36.92320755]] vector [98.0734636  18.40347527 36.73108338]


knockoff: [[73.94328901 20.23489104 48.4326329 ]] vector [72.76158358 20.83426172 48.66220447]


knockoff: [[86.05856521 10.0574667   7.80082916]] vector [85.57016487  9.90305372  7.57492661]


knockoff: [[

knockoff: [[107.47156331 -11.29594571 -28.8712467 ]] vector [107.57867422 -11.37799321 -28.89706837]


knockoff: [[ 92.43735405 -15.26129057 -35.01544022]] vector [ 92.31913214 -14.92873118 -35.39096171]


knockoff: [[195.03689374  17.17888185  38.50913042]] vector [195.27209472  16.90192682  38.66945686]


knockoff: [[ 63.1913832  -10.72358328 -25.24251749]] vector [ 63.76950754  -9.39687846 -24.02468202]


knockoff: [[156.26639727   2.18322269   6.3989947 ]] vector [156.44483948   2.53831474   6.54066003]


knockoff: [[ 95.69489408  -8.81854318 -39.81723357]] vector [ 94.91737322  -8.69214257 -40.05280576]


knockoff: [[122.35036079  -7.75548954 -14.86917257]] vector [121.94963356  -8.25993956 -14.82538661]


knockoff: [[ 80.50931247 -11.25146814 -36.1008147 ]] vector [ 80.57343734 -11.62272622 -36.61510287]


knockoff: [[129.77716508  -9.07238743 -28.73545308]] vector [128.40781228  -8.76593917 -28.45863533]


knockoff: [[104.51619102 -14.60876385 -43.85903613]] vector [104.64158102